The following codes show how the topics dicussed in the lecture may be applied to problems. 

## Example 1: Optimizing City Grid Layout with Local Search

Local search algorithms, like hill climbing, offer approaches for solving optimization problems by iteratively searching for better solutions in the neighborhood of the current state. They are often useful for dealing with situations in which due to vastness of the state space, it's impossible to navigate all the options to conclusively say which option is the best. This example uses a hill climbing local search algorithm to optimize the layout of a city grid, focusing on the strategic placement of hospitals to serve houses efficiently.

### Problem Statement

Given a city grid with specified dimensions, the objective is to place a predetermined number of hospitals in a way that minimizes the total distance between houses and their nearest hospital. This problem involves generating an initial random layout of houses within the grid and then iteratively relocating hospitals to improve the overall "cost" - the summed distance from each house to its closest hospital.

### Implementation Details

- The city grid is randomly populated with houses, and an initial set of hospitals is placed.
- The hill climbing algorithm seeks to minimize the total cost by exploring neighboring states - different hospital placements - and choosing the move that results in the lowest cost.
- An optional random restart mechanism can be employed to improve the solution by avoiding local minima traps.


In [7]:
import random


class Space():

    def __init__(self, height, width, num_hospitals):
        """Create a new state space with given dimensions."""
        self.height = height
        self.width = width
        self.num_hospitals = num_hospitals
        self.houses = set()
        self.hospitals = set()

    def add_house(self, row, col):
        """Add a house at a particular location in state space."""
        self.houses.add((row, col))

    def available_spaces(self):
        """Returns all cells not currently used by a house or hospital."""

        # Consider all possible cells
        candidates = set(
            (row, col)
            for row in range(self.height)
            for col in range(self.width)
        )

        # Remove all houses and hospitals
        for house in self.houses:
            candidates.remove(house)
        for hospital in self.hospitals:
            candidates.remove(hospital)
        return candidates

    def hill_climb(self, maximum=None, image_prefix=None, log=False):
        """Performs hill-climbing to find a solution."""
        count = 0

        # Start by initializing hospitals randomly
        self.hospitals = set()
        for i in range(self.num_hospitals):
            self.hospitals.add(random.choice(list(self.available_spaces())))
        if log:
            print("Initial state: cost", self.get_cost(self.hospitals))
        if image_prefix:
            self.output_image(f"{image_prefix}{str(count).zfill(3)}.png")

        # Continue until we reach maximum number of iterations
        while maximum is None or count < maximum:
            count += 1
            best_neighbors = []
            best_neighbor_cost = None

            # Consider all hospitals to move
            for hospital in self.hospitals:

                # Consider all neighbors for that hospital
                for replacement in self.get_neighbors(*hospital):

                    # Generate a neighboring set of hospitals
                    neighbor = self.hospitals.copy()
                    neighbor.remove(hospital)
                    neighbor.add(replacement)

                    # Check if neighbor is best so far
                    cost = self.get_cost(neighbor)
                    if best_neighbor_cost is None or cost < best_neighbor_cost:
                        best_neighbor_cost = cost
                        best_neighbors = [neighbor]
                    elif best_neighbor_cost == cost:
                        best_neighbors.append(neighbor)

            # None of the neighbors are better than the current state
            if best_neighbor_cost >= self.get_cost(self.hospitals):
                return self.hospitals

            # Move to a highest-valued neighbor
            else:
                if log:
                    print(f"Found better neighbor: cost {best_neighbor_cost}")
                self.hospitals = random.choice(best_neighbors)

            # Generate image
            if image_prefix:
                self.output_image(f"{image_prefix}{str(count).zfill(3)}.png")

    def random_restart(self, maximum, image_prefix=None, log=False):
        """Repeats hill-climbing multiple times."""
        best_hospitals = None
        best_cost = None

        # Repeat hill-climbing a fixed number of times
        for i in range(maximum):
            hospitals = self.hill_climb()
            cost = self.get_cost(hospitals)
            if best_cost is None or cost < best_cost:
                best_cost = cost
                best_hospitals = hospitals
                if log:
                    print(f"{i}: Found new best state: cost {cost}")
            else:
                if log:
                    print(f"{i}: Found state: cost {cost}")

            if image_prefix:
                self.output_image(f"{image_prefix}{str(i).zfill(3)}.png")

        return best_hospitals

    def get_cost(self, hospitals):
        """Calculates sum of distances from houses to nearest hospital."""
        cost = 0
        for house in self.houses:
            cost += min(
                abs(house[0] - hospital[0]) + abs(house[1] - hospital[1])
                for hospital in hospitals
            )
        return cost

    def get_neighbors(self, row, col):
        """Returns neighbors not already containing a house or hospital."""
        candidates = [
            (row - 1, col),
            (row + 1, col),
            (row, col - 1),
            (row, col + 1)
        ]
        neighbors = []
        for r, c in candidates:
            if (r, c) in self.houses or (r, c) in self.hospitals:
                continue
            if 0 <= r < self.height and 0 <= c < self.width:
                neighbors.append((r, c))
        return neighbors

    def output_image(self, filename):
        """Generates image with all houses and hospitals."""
        from PIL import Image, ImageDraw, ImageFont
        cell_size = 100
        cell_border = 2
        cost_size = 40
        padding = 10

        # Create a blank canvas
        img = Image.new(
            "RGBA",
            (self.width * cell_size,
             self.height * cell_size + cost_size + padding * 2),
            "white"
        )
        house = Image.open("assets/images/House.png").resize(
            (cell_size, cell_size)
        )
        hospital = Image.open("assets/images/Hospital.png").resize(
            (cell_size, cell_size)
        )
        font = ImageFont.truetype("assets/fonts/OpenSans-Regular.ttf", 30)
        draw = ImageDraw.Draw(img)

        for i in range(self.height):
            for j in range(self.width):

                # Draw cell
                rect = [
                    (j * cell_size + cell_border,
                     i * cell_size + cell_border),
                    ((j + 1) * cell_size - cell_border,
                     (i + 1) * cell_size - cell_border)
                ]
                draw.rectangle(rect, fill="black")

                if (i, j) in self.houses:
                    img.paste(house, rect[0], house)
                if (i, j) in self.hospitals:
                    img.paste(hospital, rect[0], hospital)

        # Add cost
        draw.rectangle(
            (0, self.height * cell_size, self.width * cell_size,
             self.height * cell_size + cost_size + padding * 2),
            "black"
        )
        draw.text(
            (padding, self.height * cell_size + padding),
            f"Cost: {self.get_cost(self.hospitals)}",
            fill="white",
            font=font
        )

        img.save(filename)


# Create a new space and add houses randomly
s = Space(height=10, width=20, num_hospitals=3)
for i in range(15):
    s.add_house(random.randrange(s.height), random.randrange(s.width))

# Use local search to determine hospital placement
hospitals = s.hill_climb(image_prefix="hospitals", log=True)

Initial state: cost 120
Found better neighbor: cost 110
Found better neighbor: cost 102
Found better neighbor: cost 98
Found better neighbor: cost 93
Found better neighbor: cost 88
Found better neighbor: cost 84
Found better neighbor: cost 81
Found better neighbor: cost 78
Found better neighbor: cost 75
Found better neighbor: cost 73
Found better neighbor: cost 71
Found better neighbor: cost 70
Found better neighbor: cost 69
Found better neighbor: cost 62
Found better neighbor: cost 57
Found better neighbor: cost 54
Found better neighbor: cost 53
Found better neighbor: cost 52
Found better neighbor: cost 51
Found better neighbor: cost 50


## Example 2: Optimizing the Production Line Through Linear Programming

Linear programming is a powerful method for optimizing various problems, including those that can be broken down into piecewise linear segments. In a standard linear programming framework, the objective is to minimize a cost function subject to a set of constraints. These constraints typically take the form of linear inequalities and define bounds on the decision variables.

### Problem Statement

**Objective:** Minimize the operational costs of two machines, X₁ and X₂, with respective costs of $50/hr and $80/hr. The cost function is defined as: 

$$
\text{Cost} = 50x₁ + 80x₂
$$

**Constraints:**

1. **Labor Constraint:** Machine X₁ requires 5 units of labor per hour, and machine X₂ requires 2 units per hour. With a total of 20 labor units available, this constraint is formalized as:

   $$
   5x₁ + 2x₂ ≤ 20
   $$

2. **Output Requirement:** To meet production needs, machine X₁ must produce 10 units per hour, and machine X₂, 12 units per hour, aiming for at least 90 units in total. Adjusting for the standard form of constraints, we have:

   $$
   -10x₁ - 12x₂ ≤ -90
   $$

This example illustrates how linear programming can be applied to optimize production efficiency by balancing costs with labor and output requirements.

In [6]:
import scipy.optimize

# Objective Function: 50x_1 + 80x_2
# Constraint 1: 5x_1 + 2x_2 <= 20
# Constraint 2: -10x_1 + -12x_2 <= -90

result = scipy.optimize.linprog(
    [50, 80],  # Cost function: 50x_1 + 80x_2
    A_ub=[[5, 2], [-10, -12]],  # Coefficients for inequalities
    b_ub=[20, -90],  # Constraints for inequalities: 20 and -90
)

if result.success:
    print(f"X1: {round(result.x[0], 2)} hours")
    print(f"X2: {round(result.x[1], 2)} hours")
else:
    print("No solution")


X1: 1.5 hours
X2: 6.25 hours


## Creating Exam Schedules Using Constraint Satisfaction Programming


Constraint Satisfaction problems are a general class of problems that are defined by a set of variable and a set of constraints where each variable has a domain of possible values and each constraint specifies the allowed combinations of values over a subset of variables. Solving CSPs often involves Backtracking search, enhanced with consistency checks to efficiently find solutions that satisfy all constraints.


as a small reminder for consistency checks dicussed in the lecture:
- **Node Consistency:** Ensures each variable can independently satisfy its unary constraints. (Not directly applied here, but relevant for understanding CSPs.)
- **Arc Consistency:** Ensures that for every pair of variables, there exists some assignment that satisfies the binary constraints between them.

### Problem Statement

This code shows a basic approach to solving a CSP without any heuristics. Program aims to assign exam dates to courses (variables `A` to `G`) in a way that all scheduling constraints are met, ensuring that no two exams that share a constraint are scheduled on the same day.


### Implementation Details

The solution employs a naive backtracking algorithm, following these steps:

1. **Check Completion:** The algorithm first checks if a complete assignment for all courses has been found.
2. **Select Unassigned Variable:** It selects the next unassigned course in a predetermined order.
3. **Assign and Check Consistency:** For each possible day (`Monday`, `Tuesday`, `Wednesday`), it assigns a day to the course and checks if this leads to a consistent schedule where no constrained courses are scheduled on the same day.
4. **Backtrack if Necessary:** If an inconsistency is found, it backtracks to try a different assignment.



In [2]:
"""
Naive backtracking search without any heuristics or inference.
"""

VARIABLES = ["A", "B", "C", "D", "E", "F", "G"]
CONSTRAINTS = [
    ("A", "B"),
    ("A", "C"),
    ("B", "C"),
    ("B", "D"),
    ("B", "E"),
    ("C", "E"),
    ("C", "F"),
    ("D", "E"),
    ("E", "F"),
    ("E", "G"),
    ("F", "G")
]


def backtrack(assignment):
    """Runs backtracking search to find an assignment."""

    # Check if assignment is complete
    if len(assignment) == len(VARIABLES):
        return assignment

    # Try a new variable
    var = select_unassigned_variable(assignment)
    for value in ["Monday", "Tuesday", "Wednesday"]:
        new_assignment = assignment.copy()
        new_assignment[var] = value
        if consistent(new_assignment):
            result = backtrack(new_assignment)
            if result is not None:
                return result
    return None


def select_unassigned_variable(assignment):
    """Chooses a variable not yet assigned, in order."""
    for variable in VARIABLES:
        if variable not in assignment:
            return variable
    return None


def consistent(assignment):
    """Checks to see if an assignment is consistent."""
    for (x, y) in CONSTRAINTS:

        # Only consider arcs where both are assigned
        if x not in assignment or y not in assignment:
            continue

        # If both have same value, then not consistent
        if assignment[x] == assignment[y]:
            return False

    # If nothing inconsistent, then assignment is consistent
    return True


solution = backtrack(dict())
print(solution)


{'A': 'Monday', 'B': 'Tuesday', 'C': 'Wednesday', 'D': 'Wednesday', 'E': 'Monday', 'F': 'Tuesday', 'G': 'Wednesday'}


### Shifting from manual implementation to using a library

The transition from Code 1 to Code 2 involves shifting from a **manual implementation of backtracking search** to using the **`constraint` library** for constraint satisfaction programming. 

- **Library Use:** Code 2 employs the `constraint` library, simplifying the problem setup and solving process.
- **Problem Setup:** Direct addition of variables and constraints using library functions, eliminating the need for custom functions to check consistency and select unassigned variables.
- **Constraint Definition:** Constraints are added using `addConstraint` with a lambda function, streamlining the process.
- **Solution Process:** The library's built-in method is used to find all solutions, removing the need for explicit backtracking logic.


In [3]:
from constraint import *

problem = Problem()

# Add variables
problem.addVariables(
    ["A", "B", "C", "D", "E", "F", "G"],
    ["Monday", "Tuesday", "Wednesday"]
)

# Add constraints
CONSTRAINTS = [
    ("A", "B"),
    ("A", "C"),
    ("B", "C"),
    ("B", "D"),
    ("B", "E"),
    ("C", "E"),
    ("C", "F"),
    ("D", "E"),
    ("E", "F"),
    ("E", "G"),
    ("F", "G")
]
for x, y in CONSTRAINTS:
    problem.addConstraint(lambda x, y: x != y, (x, y))

# Solve problem
for solution in problem.getSolutions():
    print(solution)


{'E': 'Wednesday', 'B': 'Tuesday', 'C': 'Monday', 'F': 'Tuesday', 'A': 'Wednesday', 'D': 'Monday', 'G': 'Monday'}
{'E': 'Wednesday', 'B': 'Monday', 'C': 'Tuesday', 'F': 'Monday', 'A': 'Wednesday', 'D': 'Tuesday', 'G': 'Tuesday'}
{'E': 'Tuesday', 'B': 'Wednesday', 'C': 'Monday', 'F': 'Wednesday', 'A': 'Tuesday', 'D': 'Monday', 'G': 'Monday'}
{'E': 'Tuesday', 'B': 'Monday', 'C': 'Wednesday', 'F': 'Monday', 'A': 'Tuesday', 'D': 'Wednesday', 'G': 'Wednesday'}
{'E': 'Monday', 'B': 'Tuesday', 'C': 'Wednesday', 'F': 'Tuesday', 'A': 'Monday', 'D': 'Wednesday', 'G': 'Wednesday'}
{'E': 'Monday', 'B': 'Wednesday', 'C': 'Tuesday', 'F': 'Wednesday', 'A': 'Monday', 'D': 'Tuesday', 'G': 'Tuesday'}
